# Backorder Prediction Using Machine Learning

## Installing Packages

In [ ]:
! pip install flask-ngrok

In [ ]:
!pip install catboost

## Libraries

In [ ]:

import pandas as pd
import numpy as np
import tqdm.notebook as tq
from scipy.sparse import hstack
from google.colab import files
from sklearn.calibration import CalibratedClassifierCV
from catboost import CatBoostClassifier
from joblib import dump, load
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings("ignore")

## Preprocessing

In [19]:
uploaded=files.upload()

Saving model.joblib to model.joblib


In [16]:
data = pd.read_csv("back_order_data.csv")

In [ ]:
def feature_extract(data):
     
     data['lead_time'] = data['lead_time'].fillna(0) #replacing nan with 0
     data['perf_6_month_avg'] = data['perf_6_month_avg'].replace(-99, 0) #replacing -99 with 0
     data['perf_12_month_avg'] = data['perf_12_month_avg'].replace(-99, 0)
     new_data = data.replace(to_replace = ["Yes", "No"], value =[1,0]) #transforming yes or no to 1 and 0 
     pm_6avg_mean = np.round(np.median(new_data.perf_6_month_avg.values),1) #finding median value of the feature
     new_data["perf_6_month_avg"].fillna(pm_6avg_mean, inplace = True) #replacing the nan with median value
     pm_12avg_mean = np.round(np.median(new_data. perf_12_month_avg.values),1)
     new_data["perf_12_month_avg"].fillna(pm_12avg_mean, inplace = True)
     

     return new_data.values

## **Final Function for Model Deployment**

In [ ]:
def final(X):
    X=X.to_frame().transpose()
    model=load('/content/model.joblib') #loading catboostclassifier model

  
    preprocessed_data=feature_extract(X)
  
    y_predictions=model.predict(preprocessed_data)
    return y_predictions

In [17]:
final(data.iloc[0])

array([0])

## Model Deployment

In [14]:
from flask import Flask, render_template, request
from flask_ngrok import run_with_ngrok 
app = Flask(__name__,template_folder='/content')
run_with_ngrok(app)    
  
@app.route("/") 
def home(): 
    return render_template("/final.html")
@app.route('/predict',methods = ['POST','GET'])
def predict():

    #loading the catboostclassifier model to do predcitions.
    model=load('/content/model.joblib')
   
    l=[]
    
    l.append(float(request.form.get('national_inv')))
    l.append(float(request.form.get('lead_time')))
    l.append(float(request.form.get('in_transit_qty')))
    l.append(float(request.form.get('forecast_3_month')))
    l.append(float(request.form.get('forecast_6_month')))
    l.append(float(request.form.get('forecast_9_month')))
    l.append(float(request.form.get('sales_1_month')))
    l.append(float(request.form.get('sales_3_month')))
    l.append(float(request.form.get('sales_6_month')))
    l.append(float(request.form.get('sales_9_month')))
    l.append(float(request.form.get('min_bank')))
    l.append(float(request.form.get('pieces_past_due')))
    
    l.append(float(request.form.get('perf_6_month_avg')))
    
    l.append(float(request.form.get('perf_12_month_avg')))
    l.append(request.form.get('local_bo_qty'))
    l.append(request.form.get('deck_risk'))
    l.append(request.form.get('oe_constraint'))
    l.append(request.form.get('ppap_risk'))
    l.append(request.form.get('stop_auto_buy'))
    
    l.append(request.form.get('rev_stop'))    
    l.append(request.form.get('potential_issue'))
    
    #creating the dataframe to do preprocessing required for our model to predict
    df=pd.DataFrame([l],columns=request.form.to_dict().keys()) #preprocessing 
    
    preprocessed_data=feature_extract(df)
    #predictions from the model.
    y_predictions=model.predict(preprocessed_data)
    if y_predictions==[0]:
        return "<h1><center><b>NOT WENT TO BACKORDER</b></center></h1>"
    else:
        return "<h1><center><b>WENT TO BACKORDER</b></center></h1>"   

app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://028edf7a52c8.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [09/Jun/2021 16:58:10] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/Jun/2021 16:58:11] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [09/Jun/2021 16:59:38] "POST /predict HTTP/1.1" 200 -


Model Deployment video : https://www.youtube.com/watch?v=fFrYhBWbJ30